In [1]:
!sudo apt-get -y -qq update


In [ ]:
# installing postgresql in collab
!sudo apt-get -y -qq install postgresql


In [3]:
# starting postgresql database server
!sudo service postgresql start


 * Starting PostgreSQL 14 database server
   ...done.


In [4]:
# alter or change the defauld postgre sql password to postgres
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"


ALTER ROLE


In [5]:
# dropping the database if it exist earlier | using below command we can run any SQL commands
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS futurex;'


NOTICE:  database "futurex" does not exist, skipping
DROP DATABASE


In [6]:
# creating a new database
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE futurex;'


CREATE DATABASE


In [7]:
#create the table in the database
# run the file psql - database name - default port - database name - sql query
!PGPASSWORD=postgres psql -q -h localhost -p 5432 -U postgres -d futurex -f create_model_table.sql


psql:create_model_table.sql:1: NOTICE:  table "futurex_model_catalog" does not exist, skipping


In [8]:
# importing python library
import psycopg2


In [9]:
# connect psycopg2 with the server
connection = psycopg2.connect(user='postgres',
                              password='postgres',
                              host='localhost',
                              database='futurex')

In [10]:
cursor = connection.cursor()


In [11]:
select_query = "select * from futurex_model_catalog"


In [12]:
cursor.execute(select_query)


In [13]:
# save all fetched records in a models variable
models = cursor.fetchall()


In [14]:
models

[]

In [16]:
# run a sample model
import numpy as np
import pandas as pd
training_data = pd.read_csv('storepurchasedata.csv')
X = training_data.iloc[:, :-1].values
y = training_data.iloc[:,-1].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
from sklearn.neighbors import KNeighborsClassifier
# minkowski is for ecledian distance
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [17]:
y_pred = classifier.predict(X_test)
y_prob = classifier.predict_proba(X_test)[:,1]

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test,y_pred))

0.875


In [18]:
# storing the model and pickle file in a binary format in a variable
import pickle

pickle_classifier_string = pickle.dumps(classifier)
pickle_sc_string = pickle.dumps(sc)

In [19]:
model_insert_sql = "INSERT INTO futurex_model_catalog VALUES(%s, %s, %s)"


In [20]:
insert_tuple = (1, 'classifier', pickle_classifier_string) # adding first row


In [21]:
cursor = connection.cursor()


In [22]:
cursor.execute(model_insert_sql, insert_tuple)


In [23]:
insert_tuple = (2, 'sc', pickle_sc_string) # adding second row


In [24]:
cursor.execute(model_insert_sql, insert_tuple)


In [25]:
# closing the connection and commit it
cursor.close()
connection.commit()

In [26]:
# lets fetch all records from the table
cursor = connection.cursor()
select_query = "select * from futurex_model_catalog"
models =cursor.execute(select_query)
models = cursor.fetchall()
models


[(1, 'classifier', <memory at 0x7ade43336140>),
 (2, 'sc', <memory at 0x7ade43336200>)]

In [27]:
# select individual model and standard scaler files
classifier_from_db = pickle.loads(models[0][2])
scaler_from_db = pickle.loads(models[1][2])

In [28]:
new_pred = classifier_from_db.predict(scaler_from_db.transform(np.array([[40,20000]])))

new_pred_proba = classifier_from_db.predict_proba(scaler_from_db.transform(np.array([[40,20000]])))[:,1]
print(new_pred)
print(new_pred_proba)

[0]
[0.2]


In [29]:
new_pred = classifier_from_db.predict(scaler_from_db.transform(np.array([[42,50000]])))

new_pred_proba = classifier_from_db.predict_proba(scaler_from_db.transform(np.array([[42,50000]])))[:,1]
print(new_pred)
print(new_pred_proba)

[1]
[0.8]
